In [75]:
import numpy as np

In [76]:
def Rx(q):
    T = np.array([[1,         0,          0, 0],
                  [0, np.cos(q), -np.sin(q), 0],
                  [0, np.sin(q),  np.cos(q), 0],
                  [0,         0,          0, 1]], dtype=float)
    return T


def Ry(q):
    T = np.array([[ np.cos(q), 0, np.sin(q), 0],
                  [         0, 1,         0, 0],
                  [-np.sin(q), 0, np.cos(q), 0],
                  [         0, 0,         0, 1]], dtype=float)
    return T


def Rz(q):
    T = np.array([[np.cos(q), -np.sin(q), 0, 0],
                  [np.sin(q),  np.cos(q), 0, 0],
                  [        0,          0, 1, 0],
                  [        0,          0, 0, 1]], dtype=float)
    return T


def Tx(x):
    T = np.array([[1, 0, 0, x],
                  [0, 1, 0, 0],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=float)
    return T


def Ty(y):
    T = np.array([[1, 0, 0, 0],
                  [0, 1, 0, y],
                  [0, 0, 1, 0],
                  [0, 0, 0, 1]], dtype=float)
    return T


def Tz(z):
    T = np.array([[1, 0, 0, 0],
                  [0, 1, 0, 0],
                  [0, 0, 1, z],
                  [0, 0, 0, 1]], dtype=float)
    return T

In [77]:
def DH(d, theta, a, alpha):
    A = Rz(theta) @ Tz(d) @ Tx(a) @ Rx(alpha)
    return A

In [91]:
# robot parameters, links (m)
l1 = 0.346
l2 = 0.324
l3 = 0.312
l4 = 1.075
l5 = 0.225
l6 = 1.280
l7 = 0.215
d6 = 0.1
q = [0,0,0,0,0,0]

In [103]:
d_p= np.array([l1, 0, 0, l3, 0, d6])
th_p = np.array([np.pi/2, 0, np.pi/2, np.pi/2, 0, 0])
a_p = np.array([0, l2, 0, 0, 0, 0])
al_p = np.array([np.pi/2, 0, np.pi/2, np.pi/2, -np.pi/2, 0])

In [80]:
def FK(q_cor):
    Matrix = np.eye(4)

    for i, q in enumerate(q_cor):
      Matrix = Matrix @ DH(d_p[i], th_p[i]+q, a_p[i], al_p[i])

    return Matrix


In [81]:
def FKold(q):
  Matrix = Tz(l1) @ Rz(q[0]) @ Tz(l2) @ Tx(l3) @ Ry(q[1]) @ Tx(l4) @ Ry(q[2]) @ Tz(l5) @ Tx(l6) @ Ry(q[3]) @ Rz(q[4]) @ Ry(q[5])

  return Matrix

In [82]:
def FK3(q):
  Matrix =  Rz(q[0]) @ Tz(l1) @ Ry(q[1]) @ Tx(l2) @ Ry(q[2]) @ Tx(l3)

  return Matrix

In [93]:
def get_cosine_law_angle(a, b, c):
  # given all sides of a triangle a, b, c
  # calculate angle gamma between sides a and b using cosine law
  cos_gamma = (a*a + b*b - c*c) / (2*a*b)
  sin_gamma = np.sqrt(1 - cos_gamma * cos_gamma)
  gamma = np.arctan2(sin_gamma, cos_gamma)

  return gamma

def first_angles(wrist_center):
    x, y, z = wrist_center
    theta1 = np.arctan2(y, x)- np.pi/2
    r = np.sqrt(x**2+y**2)
    h = z - l1
    p = np.sqrt(r**2+h**2)
    alpha = np.arctan2(h, r)
    gamma1 = get_cosine_law_angle(l2, p, l3)
    theta2 = alpha - gamma1
    gamma2 = get_cosine_law_angle(l2, l3, p)
    theta3 = np.pi - gamma2
    return theta1, theta2, theta3


def get_wrist_center(trans):
    position_end = trans[:3,3] # take the position of the end effector
    z_vec = trans[:3,2] # take the z-vecotr of the last 2 frames
    print(z_vec)
    wrist_center = position_end - d6 * z_vec
    return wrist_center



In [105]:
q = [0,  0, 0, 0, 0, 0]
print(FK(q[:6]))

[[ 1.00000000e+00  1.22464680e-16  1.22464680e-16  7.02947263e-17]
 [-1.22464680e-16 -6.12323400e-17  1.00000000e+00  7.36000000e-01]
 [ 1.22464680e-16 -1.00000000e+00 -6.12323400e-17  3.46000000e-01]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]


In [106]:
q = [0,  0, 0, 0, 0, 0]
FK(q[:4])[:3,3] - get_wrist_center(FK(q))

[ 1.2246468e-16  1.0000000e+00 -6.1232340e-17]


array([0., 0., 0.])

In [85]:
q = [np.pi/5,  np.pi/8, np.pi/6, 0, np.pi/5, np.pi/8]
print(q)
first_angles(FK(q)[:3,3])

[0.6283185307179586, 0.39269908169872414, 0.5235987755982988, 0, 0.6283185307179586, 0.39269908169872414]


(0.6283185307179586, 0.3926990816987245, 0.5235987755982987)

In [40]:
FK(q)

array([[-6.12323400e-17,  1.00000000e+00,  1.23259516e-32,
         1.98392781e-17],
       [-1.00000000e+00, -6.12323400e-17,  1.22464680e-16,
         3.24000000e-01],
       [ 1.22464680e-16,  0.00000000e+00,  1.00000000e+00,
         3.46000000e-01],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         1.00000000e+00]])

In [ ]:
q= [0, np.pi/6, 0]

In [ ]:
FK3(q)

array([[ 0.8660254 ,  0.        ,  0.5       ,  0.55079216],
       [ 0.        ,  1.        ,  0.        ,  0.        ],
       [-0.5       ,  0.        ,  0.8660254 ,  0.028     ],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])

In [ ]:
FK3(first_angles(FK3(q)[:3,3]))

array([[ 0.86591962, -0.5       ,  0.01353576,  0.47165243],
       [ 0.49993892,  0.8660254 ,  0.00781487,  0.27230866],
       [-0.01562975,  0.        ,  0.99987785,  0.56661732],
       [ 0.        ,  0.        ,  0.        ,  1.        ]])